In [1]:
import pandas as pd
import requests
# import json

file_path = 'pokedex_history.hdf5'

df_pokedex = pd.read_hdf(file_path)

print(df_pokedex.head())

# Initialize a DataFrame for Pokémon data
df_pokemon = pd.DataFrame(columns=['pokemon', 'hp', 'attack', 'defense', 'special_attack', 'special_defense', 'speed'])


         meeting_date       name
0 2015-07-15 04:15:23   Slurpuff
1 2015-10-22 23:28:34    Scrafty
2 2015-11-07 14:14:39  Mandibuzz
3 2015-12-23 08:51:11      Yanma
4 2016-03-04 06:50:08     Comfey


In [2]:
base_url = 'https://pokeapi.co/api/v2/pokemon/'

pokemon_name = 'pikachu'
pokemon_name = pokemon_name.lower()
response = requests.get(base_url + pokemon_name)
response = response.json()
response['stats']


[{'base_stat': 35,
  'effort': 0,
  'stat': {'name': 'hp', 'url': 'https://pokeapi.co/api/v2/stat/1/'}},
 {'base_stat': 55,
  'effort': 0,
  'stat': {'name': 'attack', 'url': 'https://pokeapi.co/api/v2/stat/2/'}},
 {'base_stat': 40,
  'effort': 0,
  'stat': {'name': 'defense', 'url': 'https://pokeapi.co/api/v2/stat/3/'}},
 {'base_stat': 50,
  'effort': 0,
  'stat': {'name': 'special-attack',
   'url': 'https://pokeapi.co/api/v2/stat/4/'}},
 {'base_stat': 50,
  'effort': 0,
  'stat': {'name': 'special-defense',
   'url': 'https://pokeapi.co/api/v2/stat/5/'}},
 {'base_stat': 90,
  'effort': 2,
  'stat': {'name': 'speed', 'url': 'https://pokeapi.co/api/v2/stat/6/'}}]

In [3]:
base_url = 'https://pokeapi.co/api/v2/pokemon/'

# join
for pokemon_name in df_pokedex['name']:
    pokemon_name = pokemon_name.lower()
    response = requests.get(base_url + pokemon_name)
    if response.status_code == 200:
        pokemon_response = response.json()

        hp              = pokemon_response['stats'][0]['base_stat']
        attack          = pokemon_response['stats'][1]['base_stat']
        defense         = pokemon_response['stats'][2]['base_stat']
        special_attack  = pokemon_response['stats'][3]['base_stat']
        special_defense = pokemon_response['stats'][4]['base_stat']
        speed           = pokemon_response['stats'][5]['base_stat']

        type_1          = pokemon_response['types'][0]['type']['name']
        
        if len(pokemon_response['types']) > 1:
            type_2      = pokemon_response['types'][1]['type']['name']
        else:
            type_2 = None

        new_pokemon_data = pd.DataFrame(data={'pokemon':        [pokemon_name], 
                                            'hp':               [hp], 
                                            'attack':           [attack], 
                                            'defense':          [defense], 
                                            'special_attack':   [special_attack], 
                                            'special_defense':  [special_defense], 
                                            'speed':            [speed],
                                            'type_1':           [type_1],
                                            'type_2':           [type_2]})

        df_pokemon = pd.concat([df_pokemon, new_pokemon_data], ignore_index=True)

print(df_pokemon.head())

     pokemon   hp attack defense special_attack special_defense speed type_1  \
0   slurpuff   82     80      86             85              75    72  fairy   
1    scrafty   65     90     115             45             115    58   dark   
2  mandibuzz  110     65     105             55              95    80   dark   
3      yanma   65     65      45             75              45    95    bug   
4     comfey   51     52      90             82             110   100  fairy   

     type_2  
0      None  
1  fighting  
2    flying  
3    flying  
4      None  


Z bazy danych odczytaj informację z kolumn against_* mówiącą o skuteczności ataku

In [4]:
df_pokemon.loc[df_pokemon['pokemon'] == 'yanma']

,pokemon,hp,attack,defense,special_attack,special_defense,speed,type_1,type_2
3,yanma,65,65,45,75,45,95,bug,flying


In [5]:

import sqlite3
import re

conn = sqlite3.connect("pokemon_against.sqlite")  # połączenie do bazy danych - pliku
cursor = conn.cursor()

pattern = re.compile(r'against_')
cursor.execute("PRAGMA table_info(against_stats)")
columns = cursor.fetchall()
col_name = [column[1] for column in columns if pattern.match(column[1])]

pokemon_name = "Pikachu"
query = f'SELECT {", ".join(col_name)} FROM against_stats WHERE name = "{pokemon_name}"'

for row in conn.execute(query):
    print(row)
conn.close()

(1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0)


In [6]:
row[idx[0]]

NameError: name 'idx' is not defined

In [ ]:
def attack_against(attacker: str, attacked: str, database: pd.DataFrame):
    pd_attacker = database.loc[database['pokemon'] == attacker.lower()].copy()
    pd_attacked = database.loc[database['pokemon'] == attacked.lower()].copy()


    if pd_attacker.loc[:, pd_attacker.columns != 'type_2'].isnull().any().any():
        print('Not enough data about attacker pokemon!')
        return None
    
    if pd_attacked.loc[:, pd_attacked.columns != 'type_2'].isnull().any().any():
        print('Not enough data about attacked pokemon!')
        return None

    conn = sqlite3.connect("pokemon_against.sqlite")  # connection to file-based database
    cursor = conn.cursor()
    pattern = re.compile(r'against_') # fetch only columns with "against_" prefix
    cursor.execute("PRAGMA table_info(against_stats)")
    columns = cursor.fetchall()
    col_name = [column[1] for column in columns if pattern.match(column[1])]

    # fetch against stats for attacker
    query = f'SELECT {", ".join(col_name)} FROM against_stats WHERE name = "{attacker.capitalize()}"'
    against_stats = conn.execute(query)
    for against_stats in conn.execute(query):
        continue
    conn.close()

    # calculate multipliers against types
    idx = [i for i, element in enumerate(col_name) if pd_attacked['type_1'].values[0] in element][0]
    against_type_1 = against_stats[idx]

    if pd_attacked['type_2'].values[0] is not None:
        idx = [i for i, element in enumerate(col_name) if pd_attacked['type_2'].values[0] in element][0]
        against_type_2 = against_stats[idx]
    else:
        against_type_2 = 1

    
    # calculate points
    attacker_points = pd_attacker['attack'].astype(float).values[0]
    attacker_points += ( pd_attacker['special_attack'].astype(float).values[0] * 1.5 )
    attacker_points *= against_type_1
    attacker_points *= against_type_2
    attacker_points *= pd_attacked['defense'].astype(float).values[0] / 100
    attacker_points *= pd_attacked['special_defense'].astype(float).values[0] / 75
    attacker_points *= (pd_attacked['speed'].astype(float).values[0] / 100) + 0.25

    if attacker_points > pd_attacked['hp'].astype(float).values[0]:
        print('Pokemon defeated!')
    else:
        print('Pokemon survived!')

    return attacker_points



attack_against('comfey', 'yanma', df_pokemon)

Pokemon survived!


28.349999999999998

In [ ]:
pokemon_type = 'fire'
idx = [i for i, element in enumerate(col_name) if pokemon_type in element][0]
# idx
col_name[idx]


'against_fire'

In [ ]:
wczytac zbior titanic, funkcja CUT do podzielenia 'age' na buckets, potem tablica przestawna, kobiety/mezczyzni klasa, liczebnosc i przetrwanie